#### Here's the manual of sqllite3
https://docs.python.org/3/library/sqlite3.html

In [2]:
import requests
import sqlite3
# get the file by requests function
url="https://data.moenv.gov.tw/api/v2/aqx_p_488?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=datacreationdate%20desc&format=JSON"
try:
    response=requests.get(url)
    response.raise_for_status()
    data=response.json()
except Exception as e:
    print(e)
else:
    # connect to sqllite3
    conn=sqlite3.connect("AQI.db")
    # execute DB by cursor
    cursor=conn.cursor()
    # insert SQL, values(?)make it possible to fill in the value
    insertSQL='''
    INSERT INTO records(sitename,county,aqi,status,pm25,date,lat,lon)
    values(?,?,?,?,?,?,?,?)
    '''
    # get the values from each item in data['records']
    for item in data['records']:
        values=(
            item['sitename'],
            item['county'],
            item['aqi'],
            item['status'],
            item['pm2.5'],
            item['datacreationdate'],
            item['latitude'],
            item['longitude']
        )
        # Create a table
        cursor.execute(insertSQL,values)
    # commit the changes and close
    conn.commit()
    cursor.close()
    conn.close()

HTTPSConnectionPool(host='data.moenv.gov.tw', port=443): Max retries exceeded with url: /api/v2/aqx_p_488?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=datacreationdate%20desc&format=JSON (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000017B10821A90>: Failed to resolve 'data.moenv.gov.tw' ([Errno 11001] getaddrinfo failed)"))


#### 查找重複值

In [5]:
# primary key is set as 'sitename' and 'date'. 
# once the collection of these two key appear more than once, it's regard as duplicate value.
conn=sqlite3.connect("AQI.db")
cursor=conn.cursor()
# define the query
query = '''
SELECT sitename, date, COUNT(*) as count
FROM records
GROUP BY sitename, date
HAVING count > 1
'''
cursor.execute(query)

#get the duplicated value
duplicates = cursor.fetchall()
for row in duplicates:
    print(f"Sitename: {row[0]}, Date: {row[1]}, Count: {row[2]}")
# close
cursor.close()
conn.close()

#### remove the duplicated value

In [6]:
conn=sqlite3.connect("AQI.db")
cursor=conn.cursor()
# select min(rowid) returns the minimum rowID of the collection of 'sitename' and 'date' as primary key
# if the rowid not in minimum of rowID, it's a duplicate data
delete_duplicates = '''
DELETE FROM records
WHERE ROWID NOT IN (
    SELECT MIN(ROWID)
    FROM records
    GROUP BY sitename, date
)
'''
# execute it
cursor.execute(delete_duplicates)
conn.commit()
# query the samples in db
cursor.execute("SELECT * FROM records")
all_records = cursor.fetchall()
print("去重後的資料長度:", len(all_records))

cursor.close()
conn.close()
# after removing the duplicates, excecute the cell of fetching duplicate again to double check if it do not contain duplicates.


去重後的資料長度: 576


In [ ]:
print(len(data['records']))
# why there's still 1000 records after removing the duplicated?
# it may get the length of JSON file, not the database

1000


### HW -wait for github to be ready